In [79]:
import cv2,os,time,glob
os.sys.path.append('../utilities/')
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
import ipywidgets as widgets
from shapely.geometry import Polygon,Point
import pandas as pd

In [80]:
logo_detector = support_utility_openvino.async_infer()
person_detector = support_utility_openvino.async_infer(ie_network=logo_detector.ie)

Available Devices :  ['CPU', 'GNA', 'GPU']
Available Devices :  ['CPU', 'GNA', 'GPU']


In [81]:
logo_detector.load_model("/home/prateek/shared_space/useful_models/logos/ssd_inception/openvino_model/frozen_inference_graph.xml","CPU")
person_detector.load_model("/home/prateek/shared_space/useful_models/openvino/person-detection-0202/FP32/person-detection-0202.xml","CPU")

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image'])
model outputs :  dict_keys(['detection_out'])


In [82]:
output_support = support_utility_openvino.create_plot()

In [83]:
data = pd.read_csv("/home/prateek/shared_space/flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt",sep=" ",header=None)
labels = data[1].unique()

In [84]:
labels

array(['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex',
       'Ferrari', 'Ford', 'Google', 'Heineken', 'HP', 'Intel',
       'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma',
       'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone',
       'Yahoo'], dtype=object)

In [85]:
images = glob.glob("sample_images/*")

In [86]:
logo_detector.reinit_model()
for im_path in images:
    img = cv2.imread(im_path)
    res = logo_detector.predict_sync(img)
    frame_op = output_support.label_obj_detection(img,res[0],labels=labels,thresh=.1)
    cv2.imshow("preview",frame_op)
    k = cv2.waitKey(0)
    cv2.imwrite(os.path.basename(im_path)+"_output.jpg",frame_op)
cv2.destroyAllWindows()
    

In [87]:
channel = "/media/prateek/shared_space/useful_vids/general/Top 5 WORST Delivery Drivers that Damaged Packages-7p497WsUEqA.mp4"
cap = cv2.VideoCapture(channel)
cap.set(1,0)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
person_detector.reinit_model()
thresh = .4

while True:
    t0 = time.time()
    ret,frame = cap.read()
#     frame = rotate_image(frame,-10)
    person_detector.predict(frame)
    if person_detector.frame_processed>=person_detector.num_requests:
        frame,attr,res = person_detector.postprocess_op()
        person,bboxes = output_support.trim_frame_with_result(frame,res[0],threshold=thresh)## person will go to bag model 

        frame_op = output_support.label_obj_detection(frame,res[0],labels=["person"],thresh=thresh)
        t1 = time.time()
        frame_op = output_support.write_text(frame_op,"FPS= %.4s"%(1/(t1-t0)),text_color=(0,0,0))

        cv2.imshow("preview",cv2.resize(frame_op,(1920,1080)))
        k = cv2.waitKey(1)
        if k==ord('q'):
            break

cv2.destroyAllWindows()
cap.release()
        

        
    
    

Internals of neural network is not easy to understand, but to create an autonomous system, multiple of such neural network have to be combined. 
What's the best way to fine tune such large meta/multi task learning?